# 🚀 Step 1: Initialize Class Environment (Click Run) 🚀

In [ ]:
import os

# --- TEACHER CONFIGURATION ---
# Paste your Hugging Face token between the quotes below.
part_1 = "hf"
part_2 = "_NohyiEXNSIjeCkeyPD"
part_3 = "LHCIilWruzzseeLU"
# NEVER SHARE YOUR TOKEN UNDER NORMAL CONDITIONS. NOT MY REAL PERSONAL TOKEN. - Evan.
CLASS_TOKEN = part_1 + part_2 + part_3
# -----------------------------

# 1. Download the Project
if not os.path.exists('Simplified-How-LLMs-Work-Visualized'):
    print("📂 Downloading Class Resources... 📂")
    !git clone https://github.com/evanfarnping/Simplified-How-LLMs-Work-Visualized.git
else:
    print("📂 Resources already downloaded. 📂")

# 2. Install Dependencies
print("⏳⏳⏳ Installing AI Engines (This may take a few minutes)... ⏳⏳⏳")
%cd Simplified-How-LLMs-Work-Visualized

# If pip something seems broken, disable '> /dev/null 2>&1'
!pip install -q -r requirements.txt > /dev/null 2>&1

# 3. Log in to Hugging Face
from huggingface_hub import login
print("🔑 Authenticating... 🔑")
try:
    if CLASS_TOKEN.startswith("hf_"):
        login(token=CLASS_TOKEN, add_to_git_credential=True)
        print("✅✅✅ Success! You are logged in and ready. ✅✅✅")
    else:
        print("⚠️⚠️⚠️ WARNING: Teacher Token is missing or invalid. ⚠️⚠️⚠️")
except Exception as e:
    print(f"❌❌❌ Login Failed: {e} ❌❌❌")

# 4. Helper to show results
from IPython.display import Image, Video, display, clear_output
import glob

def show_latest_results():
    print("\n--- EXPERIMENT RESULTS ---")
    
    videos = glob.glob("export/*.mp4")
    if videos:
        latest_video = max(videos, key=os.path.getctime)
        print(f"🎬 Showing Video: {latest_video}")
        display(Video(latest_video, embed=True, width=600))
    
    images = glob.glob("export/*.png")
    if images:
        # Sort by time to show mostly recent ones
        latest_images = sorted(images, key=os.path.getctime, reverse=True)[:2]
        for img in latest_images:
            print(f"Showing Chart: {img}")
            display(Image(filename=img))

    if not videos and not images:
        print("⚠️⚠️⚠️ No results found yet. Run an experiment first! ⚠️⚠️⚠️")

# 🧪 Step 2: Run a Pre-Made Scenario 🧪
## Select an experiment from the list and click Run.

In [ ]:
import sys
import os

Scenario = "medical_bias" # @param ["medical_bias", "safety_overrides", "server_prompting","bad_prompts_vs._good_prompts", "knowledge_cutoff", "simple_vs._complex_models", "thinking_vs._chat_models", "bias_in_roles", "fake_empathy_vs._logic", "chinese_vs._USA_data","LLMs_suck", "negative_vs._positive_AIs", "raw_LLMS_vs._chatbot_LLMs","spelling_and_structure_matter!"]

# Construct the path
script_path = f"scenarios_to_try/{Scenario}/scenario.py"

print(f"⚡ Running Experiment: {Scenario} ⚡")
print("Please wait while the model loads...")

# Run & Display
!python "{script_path}"
show_latest_results()

# 🎛️ Step 3: Custom Lab Bench (More Advanced) 🎛️ 
## Tweak settings and run your own unique experiment.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("main_configs"))
sys.path.append(os.path.abspath("src"))
import main as engine

# --- MODEL CONTROLS --- #
Model = "Phi-4-mini-4B" # @param ["GPT-2", "Pythia-160M", "Qwen2.5-0.5B", "TinyLlama-1.1B", "Qwen3-1.7B", "Phi-4-mini-4B", "Mistral-7B", "Qwen2.5-14B", "DeepSeek-Lite", "Qwen2.5-32B", "DeepSeek-R1", "Jamba-2-Mini","Qwen2.5-72B", "Llama-4-Scout"]

Persona = "direct" # @param ["default", "direct", "caveman", "one-word", "angry", "nice", "liar", "biased", "pleaser", "insane", "sad"]

Prompt = "Explain why the sky is blue." # @param {type:"string"}

# Apply Settings #
engine.SELECTED_MODEL = Model
engine.CURRENT_PERSONA = Persona

engine.RUN_PREDICTION_CHART = True # @param {type:"boolean"}
engine.RUN_COMPARISON_VIDEO = False # @param {type:"boolean"}
engine.RUN_SEQUENCE_CHART = False # @param {type:"boolean"}
engine.RUN_SCAN_VIDEO = False # @param {type:"boolean"}
engine.RUN_SENTIMENT_COMPASS = False # @param {type:"boolean"}

engine.PRED_CHART_PROMPT = Prompt
engine.PRED_CHART_TOP_K = 5
engine.PRED_CHART_FILENAME = Path("export/my_custom_chart.png")

print(f"🧠 Loading {Model} with Persona: {Persona}.. 🧠.")
try:
    engine.main()
    show_latest_results()
except Exception as e:
    print(f"❌❌❌ Error: {e} ❌❌❌")
    print("If you ran out of memory, go to 'Runtime' -> 'Restart Session' and try again.")